In [ ]:
import cv2
import numpy as np

# Load the video
video = cv2.VideoCapture('birdi.mp4')

# Read the first frame
ret, prev_frame = video.read()
if not ret:
    print("Error: Unable to read the video.")
    exit()

# Convert the first frame to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Parameters for Lucas-Kanade Optical Flow
lk_params = dict(
    winSize=(15, 15),  # Window size
    maxLevel=5,        # Level of scale (trajectory)
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.3)
)

# Detect good features to track
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask=None, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Create a mask for drawing
mask = np.zeros_like(prev_frame)

while True:
    ret, curr_frame = video.read()
    if not ret:
        break

    # Convert the current frame to grayscale
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow using Lucas-Kanade method
    curr_corners, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_corners, None, **lk_params)

    # Select the good points where flow was found
    good_old = prev_corners[status == 1]
    good_new = curr_corners[status == 1]

    # Draw the tracks
    for (old, new) in zip(good_old, good_new):
        x_old, y_old = old.ravel()
        x_new, y_new = new.ravel()
        mask = cv2.line(mask, (int(x_old), int(y_old)), (int(x_new), int(y_new)), (0, 255, 0), 2)
        curr_frame = cv2.circle(curr_frame, (int(x_new), int(y_new)), 5, (0, 0, 255), -1)

    # Combine the current frame with the mask
    output = cv2.add(curr_frame, mask)

    # Display the result
    cv2.imshow('Lucas-Kanade Optical Flow', output)

    # Exit when the 'q' key is pressed
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    # Update the previous frame and points
    prev_gray = curr_gray.copy()
    prev_corners = good_new.reshape(-1, 1, 2)

# Release the video capture object and close all OpenCV windows
video.release()
cv2.destroyAllWindows()